# Spark SQL

## 1. Introduction to DataFrames and Datasets

### Difference between RDDs, DataFrames and DataSets.

**RDDs (2011)**:
- Distribute collection of Java Virtual Machine (JVM) objects
- Functional Operators (map, reduce, filter, etc)

**DataFrame (2013)**:
- Distributed collection of Row objects
- Expression based operations and User Defined Functions (UDFs)
- Logical plans and optimizer
- Fast/efficient internal representations

**DataSet (2015)**:
- Internally Rows, externally JVM objects
- Almost the "Best of both worlds": type safe + fast
- Slower than DataFrame, not as good for interactive analysis, especially Python

## 2. DataFrames

<img src="images/dataframe_concept.png" title="DataFrame Concept" width="700px"/>

### Basic concepts
- Distributed collection of Row objects. These objects contain the schema within the data.
- Data is organized into columns like a relational database.

The main features of Dataframes are:
- **Catalyst**: powers the Dataframe and SQL APIs.
    1. Analyzing a logical plan to resolve references
    2. Logical plan optimization
    3. Physical planning
    4. Code generation to compile parts of the query to Java bytecode.
- **Tungsten**: provides a physical execution backend which explicitly manages memory and dynamically generates bytecode for expression evaluation.


### Creating DataFrames

DataFrames can be created from many different sources such as existing RDDs, structured and unstructured data files (CSV, JSON, Parquet), databases using JDBC, etc.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import Row, IntegerType

# Initialize Spark session
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

# Creating DataFrame from an existing RDD
rdd = spark.sparkContext.parallelize([0, 1, 2, 3, 4, 5])
df = rdd.map(lambda x: Row(x)).toDF()
print("DataFrame from RDD.toDF() function")
df.show()
schema = IntegerType()
df = spark.createDataFrame(rdd, schema)
print("DataFrame from RDD with saprk.createDataFrame()")
df.show()

# Creating a DataFrame from a Python collection
cars = [
    {"brand": "Toyota", "name": "Camry", "price": 24000},
    {"brand": "Honda", "name": "Civic", "price": 22000},
    {"brand": "Ford", "name": "Mustang", "price": 27000},
    {"brand": "Tesla", "name": "Model 3", "price": 35000},
    {"brand": "Chevrolet", "name": "Malibu", "price": 23000}
]
df = spark.createDataFrame(cars)
print("DataFrame from a Python collection")
df.show()

# Creating a DataFrame from a JSON file
df = spark.read.json('datasets/students.json')
print("DataFrame from a JSON file")
df.show()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/15 14:03:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/15 14:03:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


DataFrame from RDD.toDF() function
+---+
| _1|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
+---+

DataFrame from RDD with saprk.createDataFrame()
+-----+
|value|
+-----+
|    0|
|    1|
|    2|
|    3|
|    4|
|    5|
+-----+

DataFrame from a Python collection
+---------+-------+-----+
|    brand|   name|price|
+---------+-------+-----+
|   Toyota|  Camry|24000|
|    Honda|  Civic|22000|
|     Ford|Mustang|27000|
|    Tesla|Model 3|35000|
|Chevrolet| Malibu|23000|
+---------+-------+-----+

DataFrame from a JSON file
+--------------------+---+--------------------+-----+--------+-----------------+
|                 _id|age|               email|grade|    name|          surname|
+--------------------+---+--------------------+-----+--------+-----------------+
|33a624e7-e6f1-40b...| 23|Valeria.Sebastian...| 7.56| Valeria| Sebastian Garcia|
|2cd47675-43f3-415...| 23|Sanchez.Abascal@g...| 8.16|    Emma|  Sanchez Abascal|
|594ea4e7-75e3-456...| 20|Sarabia.Lopez@gma...| 8.22| Agustin|    Sarabia

## 3. Running SQL Queries using Spark SQL

In Spark you can work with DataFrames using the built-in functions or the Spark SQL API that provides a syntax similar to standard SQL but includes many extra ones that can be found [here](https://spark.apache.org/docs/latest/api/sql/index.html) (I recommend you to add this page to your bookmarks!). 

`NOTE`: Depending on the Spark version you are using this functions may change, you can find all the available Spark Docs for the different versions [here](https://spark.apache.org/docs/). To check you Spark version print your Spark Session.

In [ ]:
spark

In [ ]:
# First, lets check the schema of the df we will work with
df.printSchema()

### Reading the top 10 lines

#### DataFrame mode

#### SQL mode

In [3]:
df.createOrReplaceTempView('students')

### Understanding the Pyshical and Logical plans

As previously stated, the **Catalyst** is in charge of creating an optimizing the plan that will be converted into a DAG and sent to the executors. The following diagram shows the steps that the Catalyst performs to produce an execution plan

<img src="images/catalyst_steps.webp" title="Catalyst Steps Diagram" width="700px"/>

In order to see the plan generated by the **Catalyst** you can call the function `explain()` which, by default, will display the Physical Plan.

#### Getting various plans
Before Apache Spark 3.0, there was only two modes available to format explain output.

- `explain(extended=False)` which displayed only the physical plan
- `explain(extended=True)` which displayed all the plans (logical and physical)

Starting from Apache Spark 3.0, you have a new parameter “mode” that produce expected format for the plan:

- `explain(mode="simple")` which will display the physical plan
- `explain(mode="extended")` which will display physical and logical plans (like "extended" option)
- `explain(mode="codegen")` which will display the java code planned to be executed
- `explain(mode="cost")` which will display the optimized logical plan and related statistics (if they exist)
- `explain(mode="formatted")` which will display a splitted output composed by a nice physical plan outline, and a section with each node details

### Top 5 students with higher grades

#### DataFrame mode

#### SQL mode

### Max and Mean grade

#### DataFrame mode

#### SQL mode

### Top 5 students with highest mean grade

#### DataFrame mode

#### SQL mode

### Average grade per student age

#### DataFrame mode

#### SQL mode

### Add column `excelent` to indicate that a student has a grade over 9.5

#### DataFrame mode

In [19]:
from pyspark.sql.functions import when, col

df.withColumn('excelent', 
      when(
          col('grade') > 9.5, 
          'YES'
      ).otherwise('NO')
    ).show()

+--------------------+---+--------------------+-----+--------+-----------------+--------+
|                 _id|age|               email|grade|    name|          surname|excelent|
+--------------------+---+--------------------+-----+--------+-----------------+--------+
|33a624e7-e6f1-40b...| 23|Valeria.Sebastian...| 7.56| Valeria| Sebastian Garcia|      NO|
|2cd47675-43f3-415...| 23|Sanchez.Abascal@g...| 8.16|    Emma|  Sanchez Abascal|      NO|
|594ea4e7-75e3-456...| 20|Sarabia.Lopez@gma...| 8.22| Agustin|    Sarabia Lopez|      NO|
|3b521244-d2d4-40b...| 25|MartinaySebastian...| 7.67| Martina|Corominas Sarabia|      NO|
|e6f52130-362f-4a5...| 19|DavidyValeria@gma...| 7.45|   David|   Miranda Grande|      NO|
|cee04454-f6ea-48b...| 20|Lopez.Bernal@outl...| 7.35|    Laia|     Lopez Bernal|      NO|
|6e5b75cd-0d5f-41f...| 22|MarcosySantiago@h...|  6.8|  Marcos|     Garcia Aznar|      NO|
|47435195-80b1-473...| 18|Judith.Garcia@gma...|  9.1|  Judith|      Garcia Cruz|      NO|
|fbdf66dc-

#### SQL mode

In [20]:
new_df = spark.sql("""
SELECT
    *,
    CASE
        WHEN grade > 9.5 THEN 'YES'
        ELSE 'NO'
    END AS excellent
FROM
    students
""")

In [22]:
from pyspark.sql.functions import count

In [25]:
new_df.groupBy('excellent') \
    .agg(count('*').alias('n_excellent')) \
    .show()

+---------+-----------+
|excellent|n_excellent|
+---------+-----------+
|      YES|        435|
|       NO|       9565|
+---------+-----------+



In [26]:
new_df.createOrReplaceTempView('temp')

In [34]:
res_df = spark.sql("""
SELECT
    excellent,
    count(*) as n_excellent
FROM
    temp
GROUP BY
    excellent
""")

In [35]:
res_df.show()

+---------+-----------+
|excellent|n_excellent|
+---------+-----------+
|      YES|        435|
|       NO|       9565|
+---------+-----------+



In [36]:
res_df.write.csv('datasets/new_dataset')

In [37]:
new_df.show()

+--------------------+---+--------------------+-----+--------+-----------------+---------+
|                 _id|age|               email|grade|    name|          surname|excellent|
+--------------------+---+--------------------+-----+--------+-----------------+---------+
|33a624e7-e6f1-40b...| 23|Valeria.Sebastian...| 7.56| Valeria| Sebastian Garcia|       NO|
|2cd47675-43f3-415...| 23|Sanchez.Abascal@g...| 8.16|    Emma|  Sanchez Abascal|       NO|
|594ea4e7-75e3-456...| 20|Sarabia.Lopez@gma...| 8.22| Agustin|    Sarabia Lopez|       NO|
|3b521244-d2d4-40b...| 25|MartinaySebastian...| 7.67| Martina|Corominas Sarabia|       NO|
|e6f52130-362f-4a5...| 19|DavidyValeria@gma...| 7.45|   David|   Miranda Grande|       NO|
|cee04454-f6ea-48b...| 20|Lopez.Bernal@outl...| 7.35|    Laia|     Lopez Bernal|       NO|
|6e5b75cd-0d5f-41f...| 22|MarcosySantiago@h...|  6.8|  Marcos|     Garcia Aznar|       NO|
|47435195-80b1-473...| 18|Judith.Garcia@gma...|  9.1|  Judith|      Garcia Cruz|       NO|

In [41]:
new_df.repartition(4).write.mode('overwrite').parquet('datasets/new_df')

## 4. Spark SQL Joins

Spark DataFrame supports all basic SQL Join Types like `INNER`, `LEFT OUTER`, `RIGHT OUTER`, `LEFT ANTI`, `LEFT SEMI`, `CROSS`, `FULL OUTER`, etc. Spark SQL Joins are wide transformations that result in data shuffling over the network hence they have huge performance issues when not designed with care.

On the other hand Spark SQL Joins comes with more optimization by default (thanks to DataFrames & Dataset) however still there would be some performance issues to consider while using them.

### Most common types of Joins

<img src="images/joins.jpg" title="Types of Joins" width="700px"/>

### How to perform a Join in Spark (cheat sheet)

1. Referencing columns with different names in each df:

```python
dfA.join(dfB, dfA["idA"] == dfB["idB"], "type-of-join")

```

2. Key column has the same alias

```python
dfA.join(dfB, on="id", how="type-of-join")

```

3. More than one key column

```python
dfA.join(dfB, ["id", "code"], "type-of-join")

dfA.join(dfB, (dfA["id"] == dfB["id"]) & (dfA["code"] == dfB["code"]), "type-of-join")

```

4. Maintaining both columns (be careful and be sure of referencing the origin of the column when using it after this)

```python
dfC = dfA.join(dfB, dfA["id"] == dfB["id"], "type-of-join")

dfC.select(dfA["id"]).show()
```

### Read the example data

In [42]:
df_emp = spark.read.json("datasets/employees.json")
df_emp.show()

+-----------+------+------+--------+------+---------------+-----------+
|emp_dept_id|emp_id|gender|    name|salary|superior_emp_id|year_joined|
+-----------+------+------+--------+------+---------------+-----------+
|         10|     1|     M|   Smith|  3000|             -1|       2018|
|         20|     2|     M|    Rose|  4000|              1|       2010|
|         10|     4|     F|   Jones|  2000|              2|       2005|
|         10|     3|     M|Williams|  1000|              1|       2010|
|         50|     6|      |   Brown|    -1|              2|       2010|
|         40|     5|      |   Brown|    -1|              2|       2010|
+-----------+------+------+--------+------+---------------+-----------+



In [43]:
df_dept = spark.read.json("datasets/departments.json")
df_dept.show()

+-------+---------+
|dept_id|dept_name|
+-------+---------+
|     10|  Finance|
|     20|Marketing|
|     30|    Sales|
|     40|       IT|
+-------+---------+



### Inner Join

In [45]:
df_emp.join(
    df_dept, 
    df_emp['emp_dept_id'] == df_dept['dept_id'], 
    how="inner"
).show()
df_emp.join(df_dept, df_emp['emp_dept_id'] == df_dept['dept_id']).show()

+-----------+------+------+--------+------+---------------+-----------+-------+---------+
|emp_dept_id|emp_id|gender|    name|salary|superior_emp_id|year_joined|dept_id|dept_name|
+-----------+------+------+--------+------+---------------+-----------+-------+---------+
|         10|     1|     M|   Smith|  3000|             -1|       2018|     10|  Finance|
|         20|     2|     M|    Rose|  4000|              1|       2010|     20|Marketing|
|         10|     4|     F|   Jones|  2000|              2|       2005|     10|  Finance|
|         10|     3|     M|Williams|  1000|              1|       2010|     10|  Finance|
|         40|     5|      |   Brown|    -1|              2|       2010|     40|       IT|
+-----------+------+------+--------+------+---------------+-----------+-------+---------+

+-----------+------+------+--------+------+---------------+-----------+-------+---------+
|emp_dept_id|emp_id|gender|    name|salary|superior_emp_id|year_joined|dept_id|dept_name|
+--------

- The employee that belongs to inexistent department with id 50 is dropped
- The Sales department that has no employees asociated is dropped

### Full Outer Join

Outer a.k.a full, fullouter join returns all rows from both Spark DataFrame, where join expression doesn’t match it returns null on respective record columns.

In [47]:
df_emp.join(df_dept, df_emp['emp_dept_id'] == df_dept['dept_id'], how="outer").show()
df_emp.join(df_dept, df_emp['emp_dept_id'] == df_dept['dept_id'], how="fullouter").show()
df_emp.join(df_dept, df_emp['emp_dept_id'] == df_dept['dept_id'], how="full").show()

+-----------+------+------+--------+------+---------------+-----------+-------+---------+
|emp_dept_id|emp_id|gender|    name|salary|superior_emp_id|year_joined|dept_id|dept_name|
+-----------+------+------+--------+------+---------------+-----------+-------+---------+
|         10|     1|     M|   Smith|  3000|             -1|       2018|     10|  Finance|
|         10|     4|     F|   Jones|  2000|              2|       2005|     10|  Finance|
|         10|     3|     M|Williams|  1000|              1|       2010|     10|  Finance|
|         20|     2|     M|    Rose|  4000|              1|       2010|     20|Marketing|
|       null|  null|  null|    null|  null|           null|       null|     30|    Sales|
|         40|     5|      |   Brown|    -1|              2|       2010|     40|       IT|
|         50|     6|      |   Brown|    -1|              2|       2010|   null|     null|
+-----------+------+------+--------+------+---------------+-----------+-------+---------+

+--------

- There is no department with id 50 hence the values for the department are null
- There are no employees in the Sales department hence the employee values are null

### Left Join

Left a.k.a Left Outer join returns all rows from the left DataFrame/Dataset regardless of match found on the right dataset when join expression doesn’t match, it assigns null for that record and drops records from right where match not found.

In [48]:
df_emp.join(df_dept, df_emp["emp_dept_id"] == df_dept["dept_id"], "left").show()

+-----------+------+------+--------+------+---------------+-----------+-------+---------+
|emp_dept_id|emp_id|gender|    name|salary|superior_emp_id|year_joined|dept_id|dept_name|
+-----------+------+------+--------+------+---------------+-----------+-------+---------+
|         10|     1|     M|   Smith|  3000|             -1|       2018|     10|  Finance|
|         20|     2|     M|    Rose|  4000|              1|       2010|     20|Marketing|
|         10|     4|     F|   Jones|  2000|              2|       2005|     10|  Finance|
|         10|     3|     M|Williams|  1000|              1|       2010|     10|  Finance|
|         50|     6|      |   Brown|    -1|              2|       2010|   null|     null|
|         40|     5|      |   Brown|    -1|              2|       2010|     40|       IT|
+-----------+------+------+--------+------+---------------+-----------+-------+---------+



- There is no department with id 50 hence the values for the department are null
- There are no employees in the Sales department hence no row is shown

### Right Join

Right a.k.a Right Outer join is opposite to left join, here it returns all rows from the right DataFrame/Dataset regardless of match found on the left dataset, when join expression doesn’t match, it assigns null for that record and drops records from left where match not found.

In [50]:
df_emp.join(df_dept, df_emp["emp_dept_id"] == df_dept["dept_id"], "right").show()
df_dept.join(df_emp, df_emp["emp_dept_id"] == df_dept["dept_id"], "left").show()

+-----------+------+------+--------+------+---------------+-----------+-------+---------+
|emp_dept_id|emp_id|gender|    name|salary|superior_emp_id|year_joined|dept_id|dept_name|
+-----------+------+------+--------+------+---------------+-----------+-------+---------+
|         10|     3|     M|Williams|  1000|              1|       2010|     10|  Finance|
|         10|     4|     F|   Jones|  2000|              2|       2005|     10|  Finance|
|         10|     1|     M|   Smith|  3000|             -1|       2018|     10|  Finance|
|         20|     2|     M|    Rose|  4000|              1|       2010|     20|Marketing|
|       null|  null|  null|    null|  null|           null|       null|     30|    Sales|
|         40|     5|      |   Brown|    -1|              2|       2010|     40|       IT|
+-----------+------+------+--------+------+---------------+-----------+-------+---------+

+-------+---------+-----------+------+------+--------+------+---------------+-----------+
|dept_id|

- There is no department with id 50 hence no row is shown
- There are no employees in the Sales department hence the values of the employee columns are null

### CrossJoin

This join combines each row of the first table with each row of the second table. For example, we have `m` rows in one table and `n` rows in another, this gives us `m * n` rows in the resulting table. 

`Note`: A table of 1000 customers combined with a table of 1000 products would produce 1,000,000 records! Try to avoid this with large tables in production.

In [51]:
df_emp.crossJoin(df_dept).show()

+-----------+------+------+--------+------+---------------+-----------+-------+---------+
|emp_dept_id|emp_id|gender|    name|salary|superior_emp_id|year_joined|dept_id|dept_name|
+-----------+------+------+--------+------+---------------+-----------+-------+---------+
|         10|     1|     M|   Smith|  3000|             -1|       2018|     10|  Finance|
|         10|     1|     M|   Smith|  3000|             -1|       2018|     20|Marketing|
|         10|     1|     M|   Smith|  3000|             -1|       2018|     30|    Sales|
|         10|     1|     M|   Smith|  3000|             -1|       2018|     40|       IT|
|         20|     2|     M|    Rose|  4000|              1|       2010|     10|  Finance|
|         20|     2|     M|    Rose|  4000|              1|       2010|     20|Marketing|
|         20|     2|     M|    Rose|  4000|              1|       2010|     30|    Sales|
|         20|     2|     M|    Rose|  4000|              1|       2010|     40|       IT|
|         

## 5. User Defined Functions (UDFs)
User Defined Functions (UDFs) in Spark allow users to define their own transformations using Python or other programming languages, and then apply those functions on a Spark DataFrame. This can be very powerful when you need to make specific transformations to your data that aren't easily achieved using Spark's built-in functions.

When a UDF is defined, under the hood, Spark serializes the function using Py4J, transfers it over to the executor nodes, and deserializes it. This allows the UDF to be executed on rows of the DataFrame in parallel. However, it's worth noting that because UDFs involve serialization and data transfer between Python and JVM, they can be considerably slower than using native Spark functions.

Here's how you can define and use a UDF in Spark with Python (PySpark):

In [106]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, StringType, LongType, ArrayType

def person(name, surname):
    return {
        "_id": "",
        "age": 15,
        "name": f'{name} {surname}',
        "details": {
            "phone_no": 10,
            "dni": 10,
            "pets": ["dog", "cat", "lion"]
        }
    }

schema = StructType(
    [
        StructField("_id", StringType(), True),
        StructField("age", LongType(), True),
        StructField("name", StringType(), True),
        StructField("details", 
            StructType(
                [
                    StructField("phone_no", LongType(), True),
                    StructField("dni", LongType(), True),
                    StructField("pets", ArrayType(StringType()), True),
                ]
            ), True
        )
    ]
)

person_udf = udf(person, schema)

#### DataFrame Mode

In [103]:
df.printSchema()

root
 |-- _id: string (nullable = true)
 |-- age: long (nullable = true)
 |-- email: string (nullable = true)
 |-- grade: double (nullable = true)
 |-- name: string (nullable = true)
 |-- surname: string (nullable = true)



In [ ]:
df.withColumn('person', person_udf('name')).select('person').show(truncate=False)

In [107]:
df.withColumn('person', person_udf('name', 'surname')).select('person').show(truncate=False)

+-------------------------------------------------------------+
|person                                                       |
+-------------------------------------------------------------+
|{, 15, Valeria Sebastian Garcia, {10, 10, [dog, cat, lion]}} |
|{, 15, Emma Sanchez Abascal, {10, 10, [dog, cat, lion]}}     |
|{, 15, Agustin Sarabia Lopez, {10, 10, [dog, cat, lion]}}    |
|{, 15, Martina Corominas Sarabia, {10, 10, [dog, cat, lion]}}|
|{, 15, David Miranda Grande, {10, 10, [dog, cat, lion]}}     |
|{, 15, Laia Lopez Bernal, {10, 10, [dog, cat, lion]}}        |
|{, 15, Marcos Garcia Aznar, {10, 10, [dog, cat, lion]}}      |
|{, 15, Judith Garcia Cruz, {10, 10, [dog, cat, lion]}}       |
|{, 15, Iker Seco Coronado, {10, 10, [dog, cat, lion]}}       |
|{, 15, Pablo Lopez Sastre, {10, 10, [dog, cat, lion]}}       |
|{, 15, Marcos Sarabia Gisbert, {10, 10, [dog, cat, lion]}}   |
|{, 15, Oriol Blanes Miranda, {10, 10, [dog, cat, lion]}}     |
|{, 15, Marcos Cuenca Lopez, {10, 10, [d

#### SQL Mode

In [91]:
spark.udf.register("person_udf", person, schema)

23/09/15 15:32:34 WARN SimpleFunctionRegistry: The function person_udf replaced a previously registered function.


<function __main__.person(name)>

In [92]:
new_df = spark.sql("""
SELECT 
    name,
    person_udf(name) as person
FROM
    students
""")

In [93]:
new_df.createOrReplaceTempView('temp')

In [94]:
new_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- person: struct (nullable = true)
 |    |-- _id: string (nullable = true)
 |    |-- age: long (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- details: struct (nullable = true)
 |    |    |-- phone_no: long (nullable = true)
 |    |    |-- dni: long (nullable = true)
 |    |    |-- pets: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)



In [98]:
spark.sql("""
SELECT 
    name,
    person.details.pets[0]
FROM
    temp
""").show(truncate=False)

+--------+----------------------+
|name    |person.details.pets[0]|
+--------+----------------------+
|Valeria |dog                   |
|Emma    |dog                   |
|Agustin |dog                   |
|Martina |dog                   |
|David   |dog                   |
|Laia    |dog                   |
|Marcos  |dog                   |
|Judith  |dog                   |
|Iker    |dog                   |
|Pablo   |dog                   |
|Marcos  |dog                   |
|Oriol   |dog                   |
|Marcos  |dog                   |
|Sandra  |dog                   |
|Lucia   |dog                   |
|Marcos  |dog                   |
|Emma    |dog                   |
|Pedro   |dog                   |
|Santiago|dog                   |
|Juan    |dog                   |
+--------+----------------------+
only showing top 20 rows



In [72]:
from pyspark.sql.types import StructType

def count_chars_and_capitalize(s):
    return {
        'num_chars': len(s),
        'all_caps': s.upper()
    }

schema = StructType(
    [
        StructField("num_chars", IntegerType(), True),
        StructField("all_caps", StringType(), True),
    ]
)

count_chars_and_capitalize_udf = udf(count_chars_and_capitalize, schema)

In [73]:
from pyspark.sql.functions import col

df.withColumn('num_chars_and_all_caps', count_chars_and_capitalize_udf(col('name'))).select('name', 'num_chars_and_all_caps').show(truncate=False)

+--------+----------------------+
|name    |num_chars_and_all_caps|
+--------+----------------------+
|Valeria |{7, VALERIA}          |
|Emma    |{4, EMMA}             |
|Agustin |{7, AGUSTIN}          |
|Martina |{7, MARTINA}          |
|David   |{5, DAVID}            |
|Laia    |{4, LAIA}             |
|Marcos  |{6, MARCOS}           |
|Judith  |{6, JUDITH}           |
|Iker    |{4, IKER}             |
|Pablo   |{5, PABLO}            |
|Marcos  |{6, MARCOS}           |
|Oriol   |{5, ORIOL}            |
|Marcos  |{6, MARCOS}           |
|Sandra  |{6, SANDRA}           |
|Lucia   |{5, LUCIA}            |
|Marcos  |{6, MARCOS}           |
|Emma    |{4, EMMA}             |
|Pedro   |{5, PEDRO}            |
|Santiago|{8, SANTIAGO}         |
|Juan    |{4, JUAN}             |
+--------+----------------------+
only showing top 20 rows

